In [1]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

In [2]:
import torch 
from tqdm import tqdm
import json
from datasets import load_dataset
from sentence_transformers import CrossEncoder

In [3]:
# get instances
dataset = load_dataset('e2e_nlg')


Using custom data configuration default
Reusing dataset e2e_nlg (/Users/garylai/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430)
100%|██████████| 3/3 [00:00<00:00, 112.09it/s]


In [4]:
dataset = dataset['train']; len(dataset)

42061

In [5]:
import random

# Write to JSON
unique_sentences = set()

Instances = []
for i in range(1000):
    # prevent duplicates
    if dataset[i]['human_reference'] in unique_sentences or dataset[i]['meaning_representation'] in unique_sentences:
        continue
    
    unique_sentences.add(dataset[i]['human_reference'])
    unique_sentences.add(dataset[i]['meaning_representation'])
    
    Instances.append({
        # "input": dataset[i]['meaning_representation'],
        # "output": [dataset[i]['human_reference']]
        "input": dataset[i]['human_reference'],
        "output": [dataset[i]['meaning_representation']]
    })

In [6]:
len(Instances)

892

In [7]:
random.shuffle(Instances)

In [9]:
dataset[-5]

{'meaning_representation': 'name[The Rice Boat], food[Indian], priceRange[cheap], customer rating[5 out of 5], area[city centre], familyFriendly[yes], near[Express by Holiday Inn]',
 'human_reference': 'The Rice Boat is a cheap Indian restaurant in the center of the city near Express by Holiday Inn. It is family friendly and has the highest customer rating.'}

In [8]:
for i, instance in enumerate(Instances):
    print(instance)
    if i > 20:
        break

{'input': 'Blue Spice is in the riverside area, it is a fast food restaurant with a moderate price range.', 'output': ['name[Blue Spice], food[Fast food], priceRange[moderate], area[riverside]']}
{'input': 'Strada provides Chinese food in the high price range. It is near Rainbow Vegetarian Café. Its customer rating is 1 out of 5.', 'output': ['name[Strada], food[Chinese], priceRange[high], customer rating[1 out of 5], familyFriendly[no], near[Rainbow Vegetarian Café]']}
{'input': 'Low priced, family friendly restaurant Fitzbillies in the riverside area rated three star.', 'output': ['name[Fitzbillies], eatType[coffee shop], food[Fast food], priceRange[cheap], customer rating[average], area[riverside], familyFriendly[yes]']}
{'input': 'Near the Express by Holiday Inn is Fitzbillies. It is moderately priced and family friendly.', 'output': ['name[Fitzbillies], priceRange[£20-25], customer rating[3 out of 5], familyFriendly[yes], near[Express by Holiday Inn]']}
{'input': 'The Twenty Two p

In [14]:
task_json = {
    "Contributors": [
        "Gary Haizhi Lai"
    ],
    "Source": [
        "e2e"
    ],
    "Categories": [
        "Text Generation"
    ],
    "Definition": "In this task, we ask you to parse restaurant descriptions into a structured data table of key-value pairs. Here are the attributes (keys) and their examples values: \n name: The Eagle,... \n eatType: restaurant, coffee shop,... \n familyFriendly: Yes / No \n priceRange: cheap, expensive,... \n food: French, Italian,... \n near: market square,... \n area: riverside, city center, ... \n customerRating: 1 of 5 (low), 4 of 5 (high), ...  \n The output table may contain all or only some of the attributes but must not contain unlisted attributes. For the output to be considered correct, it also must parse all of the attributes existant in the input sentence; in other words, incomplete parsing would be considered incorrect.",
    "Positive Examples": [
        {
            "input": "Aromi is an English restaurant in the city centre.",
            "output": "name[Aromi], eatType[restaurant], food[English], area[city centre]",
            "explanation": "The output correctly parses all the parseable attributes in the input, no more, no less."
        },
        {
            "input": "The Rice Boat is a cheap Indian restaurant in the center of the city near Express by Holiday Inn. It is family friendly and has the highest customer rating.",
            "output": "name[The Rice Boat], food[Indian], priceRange[cheap], customer rating[5 out of 5], area[city centre], familyFriendly[yes], near[Express by Holiday Inn]",
            "explanation": "The output data table contains all the correct attributes and values from the input. All the attributes are as listed in the instruction; there are no made-up attributes."
        },
    ],
    "Negative Examples": [
        {
            "input": "Blue Spice is a coffee shop located by the riverside, near Avalon. Its prices are over £30. Its customer ratings are 5 out of 5.",
            "output": "name[Blue Spice], eatType[pub], priceRange[more than £30], customer rating[5 out of 5], area[Boston], familyFriendly[yes], near[Avalon]",
            "explanation": "The values for some of the attributes are incorrect e.g. \"pub\" and \"Boston\"."
        },
        {
            "input": "The Waterman is an expensive family-friendly Japanese restaurant with average customer rating in riverside",
            "output": "name[The Waterman], customer rating[average], area[riverside], familyFriendly[yes]",
            "explanation": "While the output correctly parses various attributes, it leaves out some parseable attributes such as \"food[Japanese]\" and \"priceRange[expensive]\"", 
        },
        {
 "input": "The Rice Boat is a cheap Indian restaurant in the center of the city near Express by Holiday Inn. It is family friendly and has the highest customer rating.",
            "output": "name[The Rice Boat], type[cheap Indian restaurant], food[Indian], priceRange[cheap], customer rating[5 out of 5], area[city centre], familyFriendly[yes], near[Express by Holiday Inn]",
            "explanation": "While most of the attributes were parsed correctly, \"type[cheap Indian restaurant]\" is a made-up attribute and therefore the output is considered incorrect."
 }
    ],
    "Instances": Instances
}

# export
with open('task950_e2e_text_generation.json', 'w') as fp:
    final_json = json.dumps(task_json, indent=4, ensure_ascii=False)
    print(final_json, file=fp)